In [3]:
import numpy as np

In [6]:

array = np.load("EEG_Data//0//101-0.npy")

print(array)


Array data: [[[2.39291960e-06]
  [2.95876372e-06]
  [1.73152924e-06]
  ...
  [1.13278249e-05]
  [3.44827929e-06]
  [6.62901109e-06]]

 [[7.14166373e-06]
  [7.52509948e-06]
  [1.24940841e-06]
  ...
  [7.60697177e-06]
  [1.14708530e-05]
  [5.74254469e-06]]

 [[2.21119028e-06]
  [5.94065952e-06]
  [4.40402931e-06]
  ...
  [3.40979912e-06]
  [8.75574838e-06]
  [1.87342256e-06]]

 ...

 [[2.80327908e-07]
  [1.58332101e-07]
  [1.24798507e-07]
  ...
  [2.72863714e-07]
  [1.58232986e-07]
  [3.57863613e-08]]

 [[2.40712227e-07]
  [1.33468023e-07]
  [1.37124951e-07]
  ...
  [1.89834042e-07]
  [1.12209757e-07]
  [1.12739671e-07]]

 [[7.87041923e-08]
  [2.03010218e-07]
  [7.80406305e-08]
  ...
  [2.00572794e-08]
  [5.65849004e-08]
  [1.11799149e-07]]]
